<a href="https://colab.research.google.com/github/deepshikharbhardwaj/Machine_Learning/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision

#Hyperparameters

In [62]:
num_classes = 10
learning_rate = 1e-3
batch_size = 1024
num_epochs = 5

# Load Pretrained Model (Pretrained = False)

In [63]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

# Load pretrain model & modify it
#model = torchvision.models.vgg16(pretrained = True)
model = torchvision.models.vgg16(pretrained = False)

# If you want to do finetuning --> TRANSFER LEARNING then set requires_grad = False
# Remove these two lines if you want to train entire model, and only want to load the pretrain weights.

#for param in model.parameters():
#    param.requires_grad = False #DOES NOT CHANGE THE WEIGHTS 

model.avgpool = Identity()
model.classifier = nn.Sequential(
                                 nn.Linear(512, 100), 
                                 nn.ReLU(), 
                                 nn.Linear(100, num_classes)  #BUT CHANGING THE MODEL FROM HERE
)
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

#Set Device

In [64]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load Data

In [65]:
train_dataset = datasets.CIFAR10(root = '/content', train = True , transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.CIFAR10(root = '/content', train = False , transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

Files already downloaded and verified
Files already downloaded and verified


#Loss and Optimizer

In [66]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

#Train Network

In [67]:
for epoch in range(num_epochs): #In one epoch model has seen all the images once
    losses = []

    for batch_idx , (data, targets) in enumerate(train_loader):
        #Get data to cuda
        data = data.to(device)
        targets = targets.to(device=device)

        #CORRECT SHAPE 
        #print(data.shape) #torch.Size([64, 1, 28, 28]) 
        #data = data.reshape(data.shape[0], -1) # untouch 0th dimension

        #forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step() #update weights based on the gradients
    print(f"Cost at epoch {epoch} is { sum(losses)/len(losses) :.5f}")

Cost at epoch 0 is 2.31018
Cost at epoch 1 is 2.30114
Cost at epoch 2 is 2.30218
Cost at epoch 3 is 2.30286
Cost at epoch 4 is 2.30270


#Check Accuracy

In [68]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking Accuracy on Training Data')
    else:
        print('Checking Accuracy on Test Data')
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            #x = x.reshape(x.shape[0], -1)

            scores = model(x)
            values, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0) #0th dimension in [64x10] i.e. 64
        print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} \n')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)



Checking Accuracy on Training Data
Got 5000/ 50000 with accuracy 10.00 

Checking Accuracy on Test Data
Got 1000/ 10000 with accuracy 10.00 





---



---



---



# Load Pretrained Model (Pretrained = True)

In [69]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

# Load pretrain model & modify it
model = torchvision.models.vgg16(pretrained = True)
#model = torchvision.models.vgg16(pretrained = False)

# If you want to do finetuning --> TRANSFER LEARNING then set requires_grad = False
# Remove these two lines if you want to train entire model, and only want to load the pretrain weights.

for param in model.parameters():
    param.requires_grad = False #DOES NOT CHANGE THE WEIGHTS 

model.avgpool = Identity()
model.classifier = nn.Sequential(nn.Linear(512, 100),
                                 nn.ReLU(),
                                 nn.Linear(100, num_classes)  #BUT CHANGING THE MODEL FROM HERE
)
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

##Train Network

In [70]:
for epoch in range(num_epochs): #In one epoch model has seen all the images once
    losses = []

    for batch_idx , (data, targets) in enumerate(train_loader):
        #Get data to cuda
        data = data.to(device)
        targets = targets.to(device=device)

        #CORRECT SHAPE 
        #print(data.shape) #torch.Size([64, 1, 28, 28]) 
        #data = data.reshape(data.shape[0], -1) # untouch 0th dimension

        #forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step() #update weights based on the gradients
    print(f"Cost at epoch {epoch} is { sum(losses)/len(losses) :.5f}")

Cost at epoch 0 is 2.32892
Cost at epoch 1 is 2.32889
Cost at epoch 2 is 2.32891
Cost at epoch 3 is 2.32894
Cost at epoch 4 is 2.32893


##Check Accuracy

In [71]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking Accuracy on Training Data')
    else:
        print('Checking Accuracy on Test Data')
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            #x = x.reshape(x.shape[0], -1)

            scores = model(x)
            values, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0) #0th dimension in [64x10] i.e. 64
        print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} \n')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)



Checking Accuracy on Training Data
Got 3956/ 50000 with accuracy 7.91 

Checking Accuracy on Test Data
Got 746/ 10000 with accuracy 7.46 

